In [1]:
import pandas as pd

In [2]:
df = pd.read_table('SMSSpamCollection.txt', header=None)
df.columns = ['y', 'text']

In [3]:
df['y'] = df.iloc[:, 0].map({'ham': 0, 'spam': 1})

In [4]:
texts = df.text
labels = df.y

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [6]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [10]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(Pipeline([("vectorizer", CountVectorizer()), 
                                    ("classifier", clf(max_iter=1000))]),
                          texts, labels, cv=10, scoring='f1').mean())
    print("\n")

<class 'sklearn.linear_model._logistic.LogisticRegression'>
0.9311542822856882


<class 'sklearn.svm._classes.LinearSVC'>
0.9412219459766616


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.9282294081136653




In [11]:
clf = Pipeline([("vectorizer", CountVectorizer()),
                ("transformer", TfidfTransformer()),
                ("classifier", LogisticRegression(max_iter=1000))])

In [12]:
clf.fit(texts, labels)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...
                ('transformer',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_s

In [13]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
        "FreeMsg: Txt: claim your reward of 3 hours talk time",
        "Have you visited the last lecture on physics?",
        "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
        "Only 99$"]

In [14]:
clf.predict(test)

array([1, 1, 0, 0, 0], dtype=int64)

In [15]:
for ngram in [(2,2), (3,3), (1,3)]:
    clf = Pipeline([("vectorizer", CountVectorizer(ngram_range=ngram)),
                    #("transformer", TfidfTransformer()),
                    ("classifier", LogisticRegression(max_iter=1000))])
    '''clf = text_classifier(CountVectorizer(ngram_range=ngram), TfidfTransformer(), LogisticRegression(max_iter=1000))'''
    score = cross_val_score(clf, texts, labels, cv=10, scoring='f1').mean()
    print(f'{ngram}  : {score:.3f}') 

(2, 2)  : 0.817
(3, 3)  : 0.725
(1, 3)  : 0.922


In [16]:
import numpy as np
import sklearn
import nltk

print(f'Pandas {pd.__version__}')
print(f'Numpy {np.__version__}')
print(f'Scikit-Learn {sklearn.__version__}')
print(f'NLTK {nltk.__version__}')

Pandas 0.25.3
Numpy 1.17.4
Scikit-Learn 0.22.1
NLTK 3.4.5


In [19]:
for ngram in [(2,2), (3,3), (1,3)]:
    X = CountVectorizer(ngram_range=ngram).fit_transform(texts)
    score = cross_val_score(MultinomialNB(), X, labels, cv=10, scoring='f1').mean()
    print(f'{ngram}  : {score:.3f}') 

(2, 2)  : 0.646
(3, 3)  : 0.379
(1, 3)  : 0.888


In [20]:
for ngram in [(1,1), (2,2), (3,3), (1,3)]:
    clf = Pipeline([("vectorizer", TfidfVectorizer(ngram_range=ngram)),
                    #("transformer", TfidfTransformer()),
                    ("classifier", MultinomialNB())])
    '''clf = text_classifier(CountVectorizer(ngram_range=ngram), TfidfTransformer(), Log"classifier",isticRegression(max_iter=1000))'''
    score = cross_val_score(clf, texts, labels, cv=10, scoring='f1').mean()
    print(f'{ngram}  : {score:.3f}') 

(1, 1)  : 0.829
(2, 2)  : 0.776
(3, 3)  : 0.665
(1, 3)  : 0.742


In [22]:
import scipy
scipy.__version__

'1.3.2'